# Back to permutation

This page goes back to permutation, now you know about [permuting
labels](fishers_tea.Rmd) and
[GroupBy](groupby.Rmd).

It gives another way of doing the permutation, that you have
already seen, as consolidation for the ideas of:

* Permutation.
* Permuting labels.
* GroupBy.

## Do mosquitoes like beer?

We have already seen this dataset in [the idea of
permutation](permutation_idea.Rmd).

If you want to run this notebook on your own computer, Download
the data from
{download}`mosquito_beer.csv <data/mosquito_beer.csv>`.

As you remember, the data are results from an experiment to test whether
drinking beer makes people more attractive to mosquitoes.

The experimenters got 43 volunteers to sit in a plastic tent. There was a
plastic tube connecting the tent to a closed box of 50 mosquitoes.  When the
subject had been sitting in the tent for long enough to let the from tent flow
down the tube, the experimenters opened the box to let the mosquitoes fly down
the tube.  There was mesh at the tent end of the tube to stop the mosquitoes
reaching the person.   For each trial, the experimenters measured the number of
mosquitoes that left the box and flew into the tube, towards the person in the
tent.  They called this count the "activated" number of mosquitoes, and took
greater counts to mean the mosquitoes were more attracted to the air in the
tent.

Of the 43 subjects, 25 drank beer before they sat in the tent, and 18 drank
water.  This page compares the "activated" counts for the 25 subjects who had
drunk beer, compared to the counts for the 18 subjects who had drunk water.

In fact each subject sat in the tent twice, once "before" they had drunk their
allocated drink (beer or water), and once "after" they had drunk their
allocated drink.

Here we only look at the "after" counts.

See [this
page](https://github.com/matthew-brett/datasets/tree/master/mosquito_beer) for
more details on the dataset, and [the data license page](data/LICENSE).

First our usual preamble:

In [ ]:
# Import Numpy library, rename as "np"
import numpy as np
# Make random number generator.
rng = np.random.default_rng()
# Import Pandas library, rename as "pd"
import pandas as pd
# Safe setting for Pandas.
pd.set_option('mode.copy_on_write', True)

# Set up plotting
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

Read in the data, select the "after" counts, and the "activated" column.

The "group" column has the allocated drink: "beer" or "water".

In [ ]:
mosquitoes = pd.read_csv('data/mosquito_beer.csv')
after_activated = mosquitoes.loc[mosquitoes['test'] == 'after',
                                 ['group', 'activated']]
after_activated

Get the means per group.

In [ ]:
means = after_activated.groupby('group')['activated'].mean()
means

Remember our procedure for *inference*:

* Find the data. Call this the real-world or *observed data*.
* Define the *measure of interest* from the data.  Call this the *statistic*.
* Calculate the statistic for the observed data.  Call this the *observed
  statistic*.
* Define the null-model — a model of the world with the effect of interest set
  to 0.
* Simulate the data in the null-model world.
* For each simulation, calculate the measure of interest - the *simulated
  statistic*.
* Use the *simulated statistics* from the simulations to build up the *sampling
  distribution* of the statistic.
* Compare the *observed statistic* to the *sampling distribution*.  Is the
  observed statistic sufficiently rare in the null-model world for us to
  question the null-model?

We will say the *data* is the `after_activated` DataFrame.

We choose the difference between the means as the *measure of interest* or
*statistic*.  We get the *observed statistic* with:

In [ ]:
observed_stat = means.loc['beer'] - means.loc['water']
observed_stat

The null-model is a world in which the average activated values for "beer" and
"water" are the same — the difference is 0.

Now we rethink how we would simulate in the null-model world.

Here is a DataFrame we can use, for which we have randomly shuffled the "group"
labels.

In [ ]:
observed_groups = after_activated['group']
fake_df = after_activated.copy()
fake_df['group'] = rng.permutation(observed_groups)
fake_df

Now we can define the procedure for one trial:

In [ ]:
fake_df['group'] = rng.permutation(observed_groups)
fake_means = fake_df.groupby('group')['activated'].mean()
fake_stat = fake_means['beer'] - fake_means['water']
fake_stat

The simulation is the trial procedure inside our standard for-loop:

In [ ]:
n_iters = 10000
fake_stats = np.zeros(n_iters)
for i in np.arange(n_iters):
    fake_df['group'] = rng.permutation(observed_groups)
    fake_means = fake_df.groupby('group')['activated'].mean()
    fake_stats[i] = fake_means['beer'] - fake_means['water']
fake_stats[:10]

Plot the sampling distribution, along with the observed value:

In [ ]:
plt.hist(fake_stats, bins=50);
plt.plot(observed_stat, 10, 'ro')
plt.title("Sampling distribution of mean difference");

The chance of any one difference being greater than the one we observed:

In [ ]:
p = np.count_nonzero(fake_stats >= observed_stat) / n_iters
p

Just for reference, a test of the same question, using the t-test:

In [ ]:
import scipy.stats as sps

by_group = after_activated.groupby('group')['activated']

sps.ttest_ind(by_group.get_group('beer'),
              by_group.get_group('water'),
              alternative='greater')

Notice the `pvalue` is very close to the permutation p-value, in this case.

With recent versions of Scipy, you can even ask Scipy to do the testing with permutations, as we have:

In [ ]:
# Do testing using permutations instead of using mathematical
# assumptions of standard t-test:
sps.ttest_ind(by_group.get_group('beer'),
              by_group.get_group('water'),
              alternative='greater',
              permutations=10000)